
# Tesis: Clasificación explicable de arritmias cardíacas desde espectrogramas (CNN + Grad-CAM)
**Autor:** Luis — **Caderno de trabajo / Pseudocódigo ejecutable**  
**Última actualización:** 2025-09-20 01:03

Este notebook define **pseudocódigo** (plantillas y funciones en blanco) para implementar el pipeline completo:
1. Datos y **split inter-paciente** (anti-leakage).
2. Segmentación en **ventanas de 5 s**.
3. Transformación a **espectrogramas** (STFT multi-resolución).
4. **Modelo** (ResNet-18 2D con opción de SE/CBAM).
5. **Entrenamiento** con pérdida ponderada / Focal Loss.
6. **Calibración** (Temperature Scaling; métricas ECE/Brier).
7. **Explicabilidad** (Grad-CAM, sanity checks, ROAR).
8. **Robustez** a ruido/artefactos (SNR 20/10 dB).
9. **Evaluación** (macro-F1, matriz de confusión, ICs).
10. **Preparación UI** (artefactos para Streamlit).

> **Nota:** Este cuaderno prioriza claridad. Muchas funciones están en *pseudocódigo* (con `pass` o comentarios) para guiar la implementación real.



## Índice
- [0. Configuración & Objetivos](#0)
- [1. Datos & Particiones inter-paciente](#1)
- [2. Segmentación & Transformaciones (STFT)](#2)
- [3. Modelo (ResNet18 + SE/CBAM)](#3)
- [4. Entrenamiento](#4)
- [5. Calibración (ECE/Brier)](#5)
- [6. Explicabilidad (Grad-CAM, ROAR)](#6)
- [7. Robustez a ruido](#7)
- [8. Evaluación & Reportes](#8)
- [9. Export para Streamlit](#9)

### Checklist anti-errores
- [ ] Split **inter-paciente** (sin sujetos repetidos).
- [ ] **AAMI mapping** fijo (N, S, V, F, Q).
- [ ] Normalizadores ajustados **solo con train**.
- [ ] CLase balanceada: pesos o **Focal Loss**.
- [ ] Semillas y versiones registradas.
- [ ] Calibración (Temperature Scaling) aplicada en test.
- [ ] XAI: sanity checks + **ROAR**.
- [ ] Robustez: SNR 20/10 dB.


## <a id='0'></a>0) Configuración & Objetivos

In [ ]:

# === 0) CONFIGURACIÓN & OBJETIVOS ===
# Este bloque define constantes y metas del experimento.

EXPERIMENT_NAME = "ecg_xai_stft_multires"
RANDOM_SEED = 42

# Objetivo principal (medible):
TARGET_MACRO_F1 = 0.90         # esperado en split inter-paciente
TARGET_ECE       = 0.05        # calibración tras temperature scaling
ACCEPTABLE_DELTA_F1_ROBUSTEZ = 0.10  # caída <=10% con SNR 10-20 dB

# Clases finales (mapeo AAMI)
CLASSES = ["N", "S", "V", "F", "Q"]

# Parámetros de segmentación
WINDOW_SECONDS = 5.0
SAMPLE_RATE = 360  # típico de MIT-BIH; ajustar si difiere

# Espectrograma (multi-res)
STFT_NFFTS = [256, 512]   # resoluciones
HOP_LENGTH = 128          # ajustar tras pruebas
WINDOW_FUNC = "hamming"   # o hann

# Entrenamiento (placeholders)
BATCH_SIZE = 64
LEARNING_RATE = 1e-3
EPOCHS = 100
EARLY_STOP_PATIENCE = 15

# Notas: Las implementaciones reales se agregarán en módulos/archivos aparte.


## <a id='1'></a>1) Datos & Particiones inter-paciente

In [ ]:

# === 1) DATOS & SPLITS ===
# Pseudocódigo: carga de MIT-BIH / PhysioNet, mapeo AAMI y split por paciente.
# Reemplazar 'pass' por código real y usar pruebas de verificación (asserts).

from typing import Dict, List, Tuple, Any

def load_mitbih_dataset(data_dir: str) -> Dict[str, Any]:
    """
    Cargar señales y anotaciones por PACIENTE (no por latido).
    Retorno sugerido:
      {
        'patients': List[str],
        'signals': Dict[patient_id, np.ndarray shape (n_samples, n_leads)],
        'ann': Dict[patient_id, List[annotations]],
        'fs': Dict[patient_id, int]  # frecuencia por paciente (si varía)
      }
    """
    # TODO: implementar con wfdb o formato disponible.
    # Importante: no mezclar info de distintos pacientes aquí.
    pass

def aami_label_map(annotation: Any) -> str:
    """
    Mapear anotación original -> etiqueta AAMI (N, S, V, F, Q).
    Documentar explícitamente el mapeo utilizado.
    """
    # TODO: implementar mapeo exacto según guía AAMI/PhysioNet.
    pass

def make_subject_splits(patients: List[str], ratios=(0.6, 0.2, 0.2)) -> Dict[str, List[str]]:
    """
    Particiona por sujeto (inter-paciente). No permitir solapamiento.
    Retorna {'train': [...], 'val': [...], 'test': [...]}
    """
    # TODO: barajar con semilla fija y dividir.
    # assert: intersección entre splits debe ser vacía.
    pass

def verify_no_leakage(splits: Dict[str, List[str]]) -> None:
    """
    Verifica que no hay pacientes repetidos entre train/val/test.
    """
    # TODO: assert de conjuntos disjuntos.
    pass

# Ejemplo de uso (pseudocódigo):
# ds = load_mitbih_dataset("/path/a/datos")
# splits = make_subject_splits(ds['patients'])
# verify_no_leakage(splits)


## <a id='2'></a>2) Segmentación & Transformaciones (STFT multi-res)

In [ ]:

# === 2) SEGMENTACIÓN & STFT ===
# Pseudocódigo de segmentación en ventanas de 5s y transformación a espectrogramas multi-res.

import numpy as np

def segment_signal(signal: np.ndarray, fs: int, window_sec: float, overlap: float=0.0) -> np.ndarray:
    """
    Corta 'signal' en ventanas de longitud window_sec con overlap opcional.
    Retorna: np.ndarray shape (num_windows, window_samples)
    """
    # TODO: implementar cortes por índices.
    pass

def window_label_majority(annotations: List[Any], window: Tuple[int, int]) -> str:
    """
    Etiquetar la ventana por mayoría de latidos (mapeados a AAMI) presentes en el rango [start, end).
    """
    # TODO: contar eventos por clase y devolver la mayoritaria (resolución de empates).
    pass

def stft_spectrogram(x: np.ndarray, n_fft: int, hop_length: int, window: str) -> np.ndarray:
    """
    Genera espectrograma (magnitud log) para una señal 1D.
    Retorna: np.ndarray (freq_bins, time_frames)
    """
    # TODO: usar biblioteca (scipy.signal o torch/STFT) en implementación real.
    pass

def multires_spectrogram(windowed_signal: np.ndarray, nffts: List[int], hop_length: int, window: str) -> np.ndarray:
    """
    Apila varias resoluciones como canales: (C, F, T)
    """
    # TODO: llamar stft_spectrogram por cada n_fft y apilar.
    pass

def build_dataset_tensors(ds: Dict[str, Any], splits: Dict[str, List[str]]) -> Tuple[Any, Any, Any]:
    """
    Construye tensores (X, y) por split:
      X: [N, C, H, W]  (C = nº resoluciones)
      y: [N]           (etiquetas AAMI)
    """
    # TODO: segmentar por paciente, etiquetar, transformar a multi-res y normalizar por ventana.
    pass

# Quick checks que debes implementar en la vida real:
# - Visualizar un espectrograma de ejemplo (matshow/imshow).
# - Verificar dimensiones: canales= len(STFT_NFFTS).
# - Confirmar distribución de clases tras segmentación.


## <a id='3'></a>3) Modelo (ResNet18 2D con opción SE/CBAM)

In [ ]:

# === 3) MODELO ===
# Pseudocódigo de arquitectura. En la implementación real, usa PyTorch y módulos existentes.

class ECGResNet18:
    def __init__(self, in_channels: int, num_classes: int, use_se: bool=True):
        """
        Inicializa una ResNet-18 2D ajustada a in_channels.
        Si use_se=True, insertar bloques Squeeze-and-Excitation (SE).
        """
        # TODO: construir backbone, adaptar primera conv a in_channels, insertar SE si aplica.
        pass

    def forward(self, x):
        """
        x: [N, C, H, W] -> logits [N, num_classes]
        """
        # TODO: forward del backbone + pool + fc.
        pass

    # Hooks para Grad-CAM (guardar activaciones y gradientes)
    def register_cam_hooks(self):
        # TODO: registrar forward/backward hooks en el último bloque conv.
        pass

def focal_loss(logits, targets, alpha_per_class, gamma=2.0):
    """
    Focal loss con pesos por clase.
    """
    # TODO: implementar focal o usar CE ponderada en la versión real.
    pass


## <a id='4'></a>4) Entrenamiento (loop, early stopping, logging)

In [ ]:

# === 4) ENTRENAMIENTO ===
# Pseudocódigo del bucle: entrenamiento/validación, early stopping por macro-F1.

def compute_class_weights(y_train: np.ndarray, classes: List[str]) -> np.ndarray:
    """
    Calcula pesos inversos a la frecuencia por clase.
    """
    # TODO: conteos y normalización.
    pass

def macro_f1(y_true, y_pred) -> float:
    """
    Calcula F1 macro.
    """
    # TODO: usar sklearn en la implementación real.
    pass

def train_one_epoch(model, train_loader, optimizer, loss_fn):
    """
    Entrena un epoch. Retorna métricas de entrenamiento.
    """
    # TODO: iterar batches, backprop, logs.
    pass

def evaluate(model, val_loader) -> Dict[str, float]:
    """
    Evalúa macro-F1, matriz de confusión provisional, etc.
    """
    # TODO: desactivar gradientes, acumular predicciones, calcular métricas.
    pass

def fit(model, train_loader, val_loader, optimizer, loss_fn, max_epochs, early_stop_patience) -> Any:
    """
    Entrena con early stopping sobre macro-F1 de validación.
    Devuelve el mejor checkpoint (o estado del modelo).
    """
    # TODO: implementar con control de paciencia y guardado del mejor.
    pass


## <a id='5'></a>5) Calibración (ECE/Brier; Temperature Scaling)

In [ ]:

# === 5) CALIBRACIÓN ===
# Pseudocódigo para ECE/Brier y temperature scaling.

def softmax(logits):
    # TODO: implementar softmax estable numéricamente.
    pass

def expected_calibration_error(probs, labels, n_bins=15) -> float:
    """
    ECE clásico por bins de confianza.
    """
    # TODO: discretizar confianzas y calcular gap promedio ponderado.
    pass

def brier_score(probs, labels) -> float:
    """
    Brier score multicategoría.
    """
    # TODO: implementar según definición.
    pass

class TemperatureScaler:
    def __init__(self):
        self.T = 1.0

    def fit(self, logits_val, labels_val):
        """
        Optimiza T para minimizar NLL en validación.
        """
        # TODO: optimización 1D de T (>0), p.ej. con grid/búsqueda o LBFGS.
        pass

    def transform_logits(self, logits):
        return logits / max(self.T, 1e-6)

# Flujo típico:
# 1) Obtener logits en val
# 2) Ajustar T
# 3) Aplicar a test y recalcular ECE/Brier


## <a id='6'></a>6) Explicabilidad (Grad-CAM, sanity checks, ROAR)

In [ ]:

# === 6) EXPLICABILIDAD ===

def grad_cam(model, x_batch) -> np.ndarray:
    """
    Genera mapas Grad-CAM para el último bloque conv.
    Retorna heatmaps con shape [N, H, W].
    """
    # TODO: extraer activaciones y gradientes, combinar y normalizar.
    pass

def sanity_checks_cam(model, x_batch):
    """
    Sanity checks de Adebayo:
      - Re-inicializar pesos y verificar que los mapas cambian.
      - Barajar etiquetas y verificar pérdida de estructura.
    """
    # TODO: implementar variantes y aserciones/plots.
    pass

def roar_fidelity_test(model, dataset, k_percent_list=[10,20,30]):
    """
    ROAR: RemOve And Retrain (o Remove And Retrain-lite si no reentrenas).
    Borrado de top-k% regiones según CAM vs borrado aleatorio; comparar F1.
    """
    # TODO: generar máscaras, borrar regiones, medir caída de F1.
    pass


## <a id='7'></a>7) Robustez a ruido/artefactos (SNR 20/10 dB)

In [ ]:

# === 7) ROBUSTEZ ===

def add_noise(signal: np.ndarray, snr_db: float) -> np.ndarray:
    """
    Añade ruido blanco para alcanzar SNR objetivo (dB).
    """
    # TODO: calcular potencia y escalar ruido.
    pass

def robustness_benchmark(model, X_test, y_test, snrs=[20, 10]):
    """
    Evalúa Δ-macro-F1 para distintos niveles de SNR.
    """
    # TODO: generar versiones ruidosas y medir F1.
    pass


## <a id='8'></a>8) Evaluación & Reportes

In [ ]:

# === 8) EVALUACIÓN & REPORTES ===

def bootstrap_confidence_intervals(y_true, y_pred, n_boot=1000, seed=42):
    """
    Intervalos de confianza por bootstrap (por paciente si es posible).
    """
    # TODO: remuestreo y percentiles (p.ej. 2.5/97.5).
    pass

def full_evaluation_report(model, test_loader, logits_val=None, labels_val=None):
    """
    1) Métricas macro-F1, por clase, matriz de confusión.
    2) Calibración (ECE/Brier) antes/después de TS.
    3) Robustez Δ-F1 (SNR).
    4) Muestras con Grad-CAM (aciertos/errores).
    5) Tiempos (preproc + infer + CAM).
    """
    # TODO: ensamblar, guardar gráficas y JSON/CSV con resultados.
    pass


## <a id='9'></a>9) Export para Streamlit (artefactos y contrato de I/O)

In [ ]:

# === 9) EXPORT PARA STREAMLIT ===

from typing import List

def export_for_streamlit(best_checkpoint_path: str, tf_params: dict, class_names: List[str]):
    """
    Guarda:
      - checkpoint del modelo
      - parámetros de TF (n_fft/hop/ventana)
      - normalizadores/estadísticos
      - etiquetas/clases
    Formato sugerido: carpeta 'artefacts/' con JSON/YAML + pesos.
    """
    # TODO: serialización con torch.save / joblib / JSON.
    pass

STREAMLIT_IO_CONTRACT = {
    "input": {
        "tipo": ["PNG escaneado", "WFDB si disponible"],
        "ventana_segundos": WINDOW_SECONDS,
        "param_tf": {"nffts": STFT_NFFTS, "hop_length": HOP_LENGTH, "window": WINDOW_FUNC}
    },
    "output": {
        "pred_clase": "string AAMI",
        "confianza_calibrada": "float [0,1]",
        "grad_cam_png": "ruta",
        "alerta_OOD": "bool",
        "reporte_pdf": "ruta opcional"
    }
}
